In [ ]:
import os
import PIL
from PIL import Image, ImageOps, ImageEnhance, __version__ as PILLOW_VERSION

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy
import re
import random
from natsort import natsorted
import glob
from skimage import img_as_ubyte
import skimage 
from skimage.color import rgb2gray
from skimage.transform import rescale, resize, downscale_local_mean
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision 
from torchvision import transforms
from torchmetrics import Dice
import cv2
import tifffile
import tifffile as tiff
from tqdm.notebook import tqdm



def create_masks(file_dir,experiment_name='CNV',first_thresh=300,thresh_small_objects=3000,thresh_big_objects=20000):

    experiment_types_dir = [  "".join([file_dir,'/',f]) for f in natsorted(os.listdir(file_dir)) if experiment_name in f]  

    for type_dir in experiment_types_dir:
        sequences=[  "".join([type_dir,'/',f])  for f in  natsorted(os.listdir(type_dir)) if experiment_name in f]
        for sequence in sequences:
            images_in_tiff = tifffile.imread(sequence)
            nb_images_in_tiff = len(images_in_tiff)
            predicted_mask=list()
            i=0         
            for index_one_image in range(nb_images_in_tiff): 
                im_original=skimage.morphology.remove_small_objects(scipy.ndimage.binary_fill_holes(skimage.morphology.binary_closing(skimage.morphology.remove_small_objects(scipy.ndimage.binary_fill_holes(skimage.filters.gabor(img_as_ubyte(skimage.exposure.rescale_intensity(images_in_tiff[index_one_image])),0.75)[1]),min_size=first_thresh, connectivity=1),footprint=np.ones((15, 15)))),min_size=thresh_small_objects,connectivity=1)
                im_big_objects=img_as_ubyte(skimage.morphology.remove_small_objects(skimage.morphology.binary_closing(  np.pad(im_original, (50, 50), 'constant', constant_values=255),footprint=np.ones((15, 15))) ,min_size=thresh_big_objects,connectivity=1)  )
                im_original=img_as_ubyte(np.pad(im_original, (50, 50), 'constant', constant_values=255) )  
                new_image = im_original - im_big_objects
                shape_im = new_image.shape
         
                if i==0:
                    predicted_mask=[new_image[50:shape_im[0]-50,50:shape_im[1]-50]] 
                else:
                    predicted_mask=np.append(predicted_mask,[new_image[50:shape_im[0]-50,50:shape_im[1]-50]],axis=0)
                i=i+1
            tiff.imsave("".join([type_dir,'/mask_',os.path.basename(sequence)]),predicted_mask)
            
    
                









In [ ]:
# Execute the function with path file 
create_masks(file_dir='/Users/aaronmamann/Desktop/test_2')